# Processing zooplankton data to DWC compliant files
## Converting a csv file to DWC headers

This notebook converts a file in the RW project data folder to a csv with column headers that match DWC metadata schema. More on DWC can be found : https://dwc.tdwg.org/terms/ 

The steps are, roughly:
1. make a new notebook, setup python pandas and other libraries as needed
2. read in the existing csv
3. look at it
4. list out the existing column names and match, as best as possible to DWC terms (the many nuances of writing metadata will come into play)
5. Rename the columns with those DWC terms
6. Write that dataframe out as a csv

In [30]:
#bring in pandas for manipulating columns, put no limit on the amount of columns in display
import pandas as pd
pd.options.display.max_columns = None

#bring in numpy to manipulate numbers
import numpy as np

## read in existing csv

In [53]:
#find the file
# %ls is a 'line magic command' for notebooks ¯\_(ツ)_/¯

%ls "../Zooplankton_505"

AMBON2015505.csv


## take a look at the file

In [54]:
#Make a dataframe
raw = pd.read_csv('../Zooplankton_505/AMBON2015505.csv', parse_dates=['Date_Time'])

raw.head()


,Cruise,Station,Type,Date_Time,Longitude_[decimal_degrees_east],Latitude_[decimal _degrees_north],Bottom_Depth_[m],Cast_Number,Depth_[m],Accepted_Organism_Identification,Life_Stage,APHIA_ID,Abundance_[#/m3],Biomass_[mg dw/m3]
0,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Acartia longiremis,,104257.0,5.882,0.0329
1,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Aglantha digitale,,117849.0,3.125,16.3242
2,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Amphipoda,,1135.0,0.138,0.1805
3,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Beroe,,1434803.0,0.138,46.6647
4,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Brachyura,zoea,106673.0,0.506,0.1321


## list existing column headers, match to DWC terms

In [55]:
# what are the existing column headers?
list(raw.columns)

['Cruise',
 'Station',
 'Type',
 'Date_Time',
 'Longitude_[decimal_degrees_east]',
 'Latitude_[decimal _degrees_north]',
 'Bottom_Depth_[m]',
 'Cast_Number',
 'Depth_[m]',
 'Accepted_Organism_Identification',
 'Life_Stage',
 'APHIA_ID',
 'Abundance_[#/m3]',
 'Biomass_[mg dw/m3]']

In [56]:
# dataframe processing

df = raw.copy()

# rename columns to DwC equivent
# https://dwc.tdwg.org/terms/#measurementorfact
#Abundance files : 'Abundance_[#/m3]': 'organismQuantity', 
#Biomass files : 'Biomass_[mg dw/m3]': 'organismQuantity' and df['organismRemarks'] = 'dry weight'

df.rename(columns={'Date_Time': 'eventDate',
                   'Depth_[m]': 'minimumDepthInMeters',
                   'Life_Stage': 'lifeStage',
                   'Bottom_Depth_[m]': 'locationRemarks',
                   'Biomass_[mg dw/m3]': 'organismQuantity',
                   'Type': 'measurementMethod',
                   'Longitude_[decimal_degrees_east]': 'decimalLongitude',
                   'Latitude_[decimal _degrees_north]': 'decimalLatitude',
                   'Accepted_Organism_Identification': 'scientificName',
                   'APHIA_ID': 'taxonID',
                   'Station': 'stationIdentifier'}, inplace=True)

# add new parameters
df['maximumDepthInMeters'] = df['minimumDepthInMeters']
df['organismQuantityType'] = 'count per cubic meter'
df['nameAccordingTo'] = 'WoRMS'
df['nameAccordingToID'] = 'https://doi.org/10.14284/170'
df['organismRemarks'] = 'dry weight'

df.head()

,Cruise,stationIdentifier,measurementMethod,eventDate,decimalLongitude,decimalLatitude,locationRemarks,Cast_Number,minimumDepthInMeters,scientificName,lifeStage,taxonID,Abundance_[#/m3],organismQuantity,maximumDepthInMeters,organismQuantityType,nameAccordingTo,nameAccordingToID,organismRemarks
0,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Acartia longiremis,,104257.0,5.882,0.0329,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
1,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Aglantha digitale,,117849.0,3.125,16.3242,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
2,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Amphipoda,,1135.0,0.138,0.1805,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
3,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Beroe,,1434803.0,0.138,46.6647,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
4,AMBON2015,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,1.0,45.0,Brachyura,zoea,106673.0,0.506,0.1321,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight


In [57]:
# Remove unnecessary columns
# In this case, the 2015 DWC data does not include the biomass measurement and abundance measurement, it's seperated into two spreadsheets

df = df.drop(columns=['Cruise', 'Cast_Number', 'Abundance_[#/m3]' ])

df.head()

,stationIdentifier,measurementMethod,eventDate,decimalLongitude,decimalLatitude,locationRemarks,minimumDepthInMeters,scientificName,lifeStage,taxonID,organismQuantity,maximumDepthInMeters,organismQuantityType,nameAccordingTo,nameAccordingToID,organismRemarks
0,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,45.0,Acartia longiremis,,104257.0,0.0329,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
1,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,45.0,Aglantha digitale,,117849.0,16.3242,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
2,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,45.0,Amphipoda,,1135.0,0.1805,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
3,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,45.0,Beroe,,1434803.0,46.6647,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight
4,DBO3.8,BONGO_505UM_MICROSCOPY,2015-08-11 21:20:00,-168.9518,67.6797,48.0,45.0,Brachyura,zoea,106673.0,0.1321,45.0,count per cubic meter,WoRMS,https://doi.org/10.14284/170,dry weight


In [58]:
list(df.columns)

['stationIdentifier',
 'measurementMethod',
 'eventDate',
 'decimalLongitude',
 'decimalLatitude',
 'locationRemarks',
 'minimumDepthInMeters',
 'scientificName',
 'lifeStage',
 'taxonID',
 'organismQuantity',
 'maximumDepthInMeters',
 'organismQuantityType',
 'nameAccordingTo',
 'nameAccordingToID',
 'organismRemarks']

## write out dataframe to new csv file

In [59]:
#save it  with abundance in the name to keep them straight
# Abundance file: df.to_csv('AMBON2015_150_Abundance_DWC.csv')
# Biomass file: df.to_csv('AMBON2015_150_Biomass_DWC.csv')

#So, when this was spitting out CSVs with quotes around teh values, Jesse recommended using to_csv(fpath, quotechar=QUOTE_NONE)` to get that to stop.
#Because https://docs.python.org/3/library/csv.html#csv-contents stuff

#But, running a test 10/29 - there's not quotes now?

df.to_csv('AMBON2015_505_Zooplankton_Biomass_DWC_again.csv')
